In [ ]:
import os
import re
import shutil
import csv
from frame_metrics import extract_frame_metrics

In [ ]:
def process_videos(source_folder, output_folder):
    # Regular expression pattern to match video files
    pattern = re.compile(r'Veep \(2012\) - S(\d{2})E(\d{2}) - (.+) \(\d+p.+\)\.mkv')

    # Loop over each season folder
    for season in range(1, 8):
        season_folder_name = f'Season {season}'
        season_folder_path = os.path.join(source_folder, season_folder_name)
        
        # Check if the season folder exists
        if not os.path.exists(season_folder_path):
            print(f"Source season folder {season_folder_path} does not exist. Skipping.")
            continue

        # Create corresponding season folder in the output directory
        output_season_path = os.path.join(output_folder, season_folder_name)
        if not os.path.exists(output_season_path):
            os.makedirs(output_season_path)
            print(f"Created folder: {output_season_path}")

        # Get a sorted list of files in the season folder
        files_in_season = sorted(os.listdir(season_folder_path))

        # Loop through each file in the sorted list
        for file_name in files_in_season:
            match = pattern.match(file_name)
            if match:
                episode_num = int(match.group(2))
                episode_title = match.group(3)
                # if episode_num != 8: continue
                
                # Define the new folder name
                new_folder_name = f'Season {season} Episode {episode_num:02d} - {episode_title}'
                episode_output_dir = os.path.join(output_season_path, new_folder_name)
                
                # Check if the directory already exists
                if not os.path.exists(episode_output_dir) or not os.listdir(episode_output_dir):
                    if not os.path.exists(episode_output_dir):
                        os.makedirs(episode_output_dir)
                        print(f"Created folder: {episode_output_dir}")

                    # Run the key frame extraction
                    video_path = os.path.join(season_folder_path, file_name)
                    frame_metrics = extract_frame_metrics(video_path)
                    csv_file_path = os.path.join(episode_output_dir, 'frame_metrics.csv')
                    with open(csv_file_path, mode='w', newline='') as csv_file:
                        # Create a CSV writer object
                        writer = csv.writer(csv_file)

                        # Write the header
                        header = ["frame_count", "ssim_overall", "ssim_top", "ssim_bottom", "histogram_similarity"]
                        writer.writerow(header)

                        # Write the data rows
                        for frame_count, metrics in frame_metrics.items():
                            row = [
                                frame_count,
                                metrics["ssim_overall"],
                                metrics["ssim_top"],
                                metrics["ssim_bottom"],
                                metrics["histogram_similarity"]
                            ]
                            writer.writerow(row)                    
                else:
                    print(f"Skipping extraction for {file_name} as folder already has the xlsx.")

In [ ]:
source_folder = "/media/coof/standardHDD/Veep"
output_folder = './VeepCSVs'
process_videos(source_folder, output_folder)
